In [17]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

In [18]:


# Constants
start_time = datetime(2023, 1, 1, 8, 0)
end_time = datetime(2023, 4, 1, 16, 0)
NUMBER_OF_CUSTOMERS = 500
NUMBER_OF_AGENTS = 20
lambda_base_weekday = 224/8/NUMBER_OF_CUSTOMERS
lambda_base_weekend = 100/8/NUMBER_OF_CUSTOMERS

# Define customer attributes
# Define customer attributes
customers = [{'id': i+1,
              'lambda_weekday': lambda_base_weekday,
              'lambda_weekend': lambda_base_weekend,
              'tolerance': timedelta(minutes=np.random.uniform(2, 8)),
              'next_possible_call_time': start_time} for i in range(NUMBER_OF_CUSTOMERS)]


def truncated_exponential_with_outlier_inverse(lambda_scale, min_val=3*60, max_val=13*60, outlier_val=25*60, outlier_prob=0.05):
    # Check for the chance of an outlier
    if np.random.random() < outlier_prob:
        return outlier_val
    
    # CDF values for the truncation limits
    cdf_min = 1 - np.exp(-min_val / lambda_scale)
    cdf_max = 1 - np.exp(-max_val / lambda_scale)
    
    # Generate a random number between cdf_min and cdf_max
    u = np.random.uniform(cdf_min, cdf_max)
    
    # Apply the inverse CDF to get the sampled value
    sampled_value = -lambda_scale * np.log(1 - u)
    
    return sampled_value


# Define agent attributes based on answer rate variability
# Define agent attributes based on answer rate variability
def create_agents(varying_answer_rate=True):
    if varying_answer_rate:
        return [{'id': i+1, 
                 'responsiveness': min(1, 0.93 + np.random.exponential(scale=0.001)),
                 'available_until': start_time,
                 'talk_duration_lambda': 5*60}  # Average of 4 minutes
                for i in range(NUMBER_OF_AGENTS)]
    else:
        return [{'id': i+1, 
                 'responsiveness': 0.93,
                 'available_until': start_time,
                 'talk_duration_lambda': 5*60} 
                for i in range(NUMBER_OF_AGENTS)]

# Training function
def train_agents(agents, method="targeted"):
    if method == "targeted":
        for agent in agents:
            # If the upper limit of talk duration is above 10 mins, apply training
            talk_duration = truncated_exponential_with_outlier_inverse(agent['talk_duration_lambda'])
            if talk_duration > 7*60:  
                if agent['responsiveness'] < 0.95:
                # Set talk duration lambda to reflect a shorter average duration
                    agent['talk_duration_lambda'] = np.random.uniform(3*60,5*60)  # Adjust as needed
                # Increase answer rate, ensuring it doesn't exceed 100% if it's below 0.95
                #if agent['answer_rate'] < 0.95:
                    agent['responsiveness'] = min(1, 0.95 + np.random.exponential(scale=0.001))
    return agents



# Function to simulate call generation using NHPP
def simulate_calls_for_customer_nhpp(customer, start_time, end_time):
    calls = []
    while True:
        if start_time.weekday() < 6:
            lambda_val = lambda_base_weekday
        else:
            lambda_val = lambda_base_weekend

        next_call_time = start_time + timedelta(hours=np.random.exponential(1 / lambda_val))
        if next_call_time <= end_time:
            calls.append({'customer_id': customer['id'], 'call_time': next_call_time, 'tolerance': customer['tolerance']})
            start_time = next_call_time
        else:
            break
    return calls
    
# Optimized simulation function with options for answer rate variability and agent training
def optimized_simulation(run, varying_answer_rate=True, training_method=None):
    # Initialize empty lists to collect call logs, answered calls, wait times, and talk durations
    call_logs, answered_calls, wait_times, talk_durations = [], [], [], []
    # Lists to store daily average wait times and talk durations for analysis
    daily_avg_wait_times, daily_avg_talk_durations = [], []
    # Set the start time for the simulation
    current_time = start_time
    
    # Define agents based on answer rate variability and apply training if specified
    agents = create_agents(varying_answer_rate=varying_answer_rate)
    if training_method == "targeted":
        agents = train_agents(agents, method="targeted")

    # Lists to store time-based metrics for analysis
    times, answer_rates, num_trained_agents = [], [], []
    
    # Run the simulation for the defined period
    while current_time <= end_time:
        # Define the end of the current day
        end_of_day = datetime(current_time.year, current_time.month, current_time.day, 16, 0)
        # Generate calls for the current day and add to call logs
        call_logs.extend([call for customer in customers for call in simulate_calls_for_customer_nhpp(customer, current_time, end_of_day)])
        
        # Process calls in chronological order
        for call in sorted(call_logs, key=lambda x: x['call_time']):
            # Find available agents at the call time
            available_agents = sorted([agent for agent in agents if agent['available_until'] <= call['call_time']], key=lambda x: x['available_until'])
            # Handle the case where no agents are available or the agent does not respond
            if not available_agents or random.random() > available_agents[0]['responsiveness']:
                # Log the call as abandoned
                answered_calls.append({'customer_id': call['customer_id'], 'call_time': call['call_time'], 'status': 'abandoned', 'agent_id': 'NA', 'call_duration': '00:00', 'wait_time': timedelta(seconds=0)})
                continue
            # Assign the call to the first available agent
            agent = available_agents[0]
            # Calculate the wait time for the call
            wait_time = min(call['tolerance'], call['call_time'] - agent['available_until'])
            # Determine the call duration using a truncated exponential distribution
            call_duration_seconds = truncated_exponential_with_outlier_inverse(agent['talk_duration_lambda'])
            # Update the agent's availability based on the call duration
            agent['available_until'] = call['call_time'] + timedelta(seconds=call_duration_seconds)
            # Log the answered call with details
            answered_calls.append({'customer_id': call['customer_id'], 'call_time': call['call_time'], 'status': 'answered', 'agent_id': agent['id'], 'call_duration': timedelta(seconds=call_duration_seconds), 'wait_time': wait_time})
            # Collect wait time and talk duration for analysis
            wait_times.append(wait_time)
            talk_durations.append(call_duration_seconds)

        # Calculate average metrics for the current day
        if wait_times:
            daily_avg_wait_times.append(np.mean(wait_times))
            daily_avg_talk_durations.append(np.mean(talk_durations))
            # Reset lists for the next day
            wait_times.clear()
            talk_durations.clear()
        else:
            # If no wait times or talk durations, append 0 for the day
            daily_avg_wait_times.append(0)
            daily_avg_talk_durations.append(0)

        # Reset call logs for the next day
        call_logs.clear()
        # Move to the start of the next working day
        current_time += timedelta(days=1)
        current_time = datetime(current_time.year, current_time.month, current_time.day, 8, 0)
        
        # Calculate answer rate at the current hour
        answered_calls_at_current_hour = [call for call in answered_calls if call['call_time'].hour == current_time.hour and call['call_time'].day == current_time.day]
        answer_rate_current_hour = len([call for call in answered_calls_at_current_hour if call['status'] == 'answered']) / len(answered_calls_at_current_hour) if answered_calls_at_current_hour else 0
        
        # Record metrics for analysis
        times.append(current_time)
        answer_rates.append(answer_rate_current_hour)
        num_trained_agents.append(sum(1 for agent in agents if agent['talk_duration_lambda'] == 3*60))
        
        # Move to the next hour for the next iteration
        current_time += timedelta(hours=1) 
        
    # Compute the overall average of daily metrics
    overall_avg_wait_time = np.mean(daily_avg_wait_times)
    overall_avg_talk_duration = np.mean(daily_avg_talk_durations)
    # Calculate the overall answer rate for the simulation run
    answer_rate_run = len([call for call in answered_calls if call['status'] == 'answered']) / len(answered_calls)

    # Return a dictionary with the computed metrics and logs
    return {
        #'average_wait_time': overall_avg_wait_time,  # Uncomment if you need this metric
        'average_talk_duration': overall_avg_talk_duration,
        'answer_rate': answer_rate_run,
        'answered_calls': answered_calls,
        'times': times,
        'answer_rates': answer_rates,
        'num_trained_agents': num_trained_agents
    }


In [19]:
# Define scenarios
scenarios = {
    "Constant Answer Rate with Standard": (False, None),
    "Constant Answer Rate with Targeted Training": (False, "targeted"),
    "Varying Answer Rate with Standard": (True, None),
    "Varying Answer Rate with Targeted Training": (True, "targeted")
}

# Number of desired runs for each scenario
num_runs = 1000

# Store results for all scenarios
all_results_reduced = {}

for name, (varying_rate, training) in scenarios.items():
    results_for_scenario = []
    for run in range(num_runs):
        result = optimized_simulation(run, varying_rate, training)
        results_for_scenario.append(result)

        # Print the results for immediate feedback
        print(f"Scenario: {name}, Simulation {run + 1}:")
        #print(f"    Average wait time: {result['average_wait_time'].total_seconds() / 60:.2f} minutes")
        print(f"    Average talk duration: {result['average_talk_duration'] / 60:.2f} minutes")
        print(f"    Answer rate: {result['answer_rate'] * 100:.2f}%")
        print("")

    all_results_reduced[name] = results_for_scenario



Scenario: Constant Answer Rate with Standard, Simulation 1:
    Average talk duration: 7.37 minutes
    Answer rate: 93.00%

Scenario: Constant Answer Rate with Standard, Simulation 2:
    Average talk duration: 7.40 minutes
    Answer rate: 92.67%

Scenario: Constant Answer Rate with Standard, Simulation 3:
    Average talk duration: 7.37 minutes
    Answer rate: 92.82%

Scenario: Constant Answer Rate with Standard, Simulation 4:
    Average talk duration: 7.35 minutes
    Answer rate: 92.94%

Scenario: Constant Answer Rate with Standard, Simulation 5:
    Average talk duration: 7.36 minutes
    Answer rate: 92.80%

Scenario: Constant Answer Rate with Standard, Simulation 6:
    Average talk duration: 7.38 minutes
    Answer rate: 92.93%

Scenario: Constant Answer Rate with Standard, Simulation 7:
    Average talk duration: 7.37 minutes
    Answer rate: 93.21%

Scenario: Constant Answer Rate with Standard, Simulation 8:
    Average talk duration: 7.30 minutes
    Answer rate: 93.13%



Scenario: Constant Answer Rate with Standard, Simulation 67:
    Average talk duration: 7.35 minutes
    Answer rate: 93.05%

Scenario: Constant Answer Rate with Standard, Simulation 68:
    Average talk duration: 7.33 minutes
    Answer rate: 93.05%

Scenario: Constant Answer Rate with Standard, Simulation 69:
    Average talk duration: 7.39 minutes
    Answer rate: 93.07%

Scenario: Constant Answer Rate with Standard, Simulation 70:
    Average talk duration: 7.37 minutes
    Answer rate: 93.05%

Scenario: Constant Answer Rate with Standard, Simulation 71:
    Average talk duration: 7.34 minutes
    Answer rate: 92.88%

Scenario: Constant Answer Rate with Standard, Simulation 72:
    Average talk duration: 7.35 minutes
    Answer rate: 93.27%

Scenario: Constant Answer Rate with Standard, Simulation 73:
    Average talk duration: 7.37 minutes
    Answer rate: 93.01%

Scenario: Constant Answer Rate with Standard, Simulation 74:
    Average talk duration: 7.40 minutes
    Answer rate: 

Scenario: Constant Answer Rate with Standard, Simulation 132:
    Average talk duration: 7.34 minutes
    Answer rate: 93.12%

Scenario: Constant Answer Rate with Standard, Simulation 133:
    Average talk duration: 7.36 minutes
    Answer rate: 93.04%

Scenario: Constant Answer Rate with Standard, Simulation 134:
    Average talk duration: 7.35 minutes
    Answer rate: 92.93%

Scenario: Constant Answer Rate with Standard, Simulation 135:
    Average talk duration: 7.39 minutes
    Answer rate: 93.33%

Scenario: Constant Answer Rate with Standard, Simulation 136:
    Average talk duration: 7.36 minutes
    Answer rate: 93.27%

Scenario: Constant Answer Rate with Standard, Simulation 137:
    Average talk duration: 7.43 minutes
    Answer rate: 93.50%

Scenario: Constant Answer Rate with Standard, Simulation 138:
    Average talk duration: 7.31 minutes
    Answer rate: 92.88%

Scenario: Constant Answer Rate with Standard, Simulation 139:
    Average talk duration: 7.37 minutes
    Answe

Scenario: Constant Answer Rate with Standard, Simulation 197:
    Average talk duration: 7.42 minutes
    Answer rate: 93.02%

Scenario: Constant Answer Rate with Standard, Simulation 198:
    Average talk duration: 7.36 minutes
    Answer rate: 93.16%

Scenario: Constant Answer Rate with Standard, Simulation 199:
    Average talk duration: 7.33 minutes
    Answer rate: 93.13%

Scenario: Constant Answer Rate with Standard, Simulation 200:
    Average talk duration: 7.35 minutes
    Answer rate: 92.88%

Scenario: Constant Answer Rate with Standard, Simulation 201:
    Average talk duration: 7.35 minutes
    Answer rate: 92.95%

Scenario: Constant Answer Rate with Standard, Simulation 202:
    Average talk duration: 7.39 minutes
    Answer rate: 92.97%

Scenario: Constant Answer Rate with Standard, Simulation 203:
    Average talk duration: 7.34 minutes
    Answer rate: 93.13%

Scenario: Constant Answer Rate with Standard, Simulation 204:
    Average talk duration: 7.36 minutes
    Answe

Scenario: Constant Answer Rate with Standard, Simulation 262:
    Average talk duration: 7.39 minutes
    Answer rate: 93.19%

Scenario: Constant Answer Rate with Standard, Simulation 263:
    Average talk duration: 7.38 minutes
    Answer rate: 93.08%

Scenario: Constant Answer Rate with Standard, Simulation 264:
    Average talk duration: 7.33 minutes
    Answer rate: 92.90%

Scenario: Constant Answer Rate with Standard, Simulation 265:
    Average talk duration: 7.34 minutes
    Answer rate: 93.16%

Scenario: Constant Answer Rate with Standard, Simulation 266:
    Average talk duration: 7.38 minutes
    Answer rate: 93.06%

Scenario: Constant Answer Rate with Standard, Simulation 267:
    Average talk duration: 7.35 minutes
    Answer rate: 92.87%

Scenario: Constant Answer Rate with Standard, Simulation 268:
    Average talk duration: 7.42 minutes
    Answer rate: 92.77%

Scenario: Constant Answer Rate with Standard, Simulation 269:
    Average talk duration: 7.40 minutes
    Answe

...

Scenario: Varying Answer Rate with Targeted Training, Simulation 739:
    Average talk duration: 7.12 minutes
    Answer rate: 94.09%

Scenario: Varying Answer Rate with Targeted Training, Simulation 740:
    Average talk duration: 7.26 minutes
    Answer rate: 93.95%

Scenario: Varying Answer Rate with Targeted Training, Simulation 741:
    Average talk duration: 7.22 minutes
    Answer rate: 93.71%

Scenario: Varying Answer Rate with Targeted Training, Simulation 742:
    Average talk duration: 7.11 minutes
    Answer rate: 94.48%

Scenario: Varying Answer Rate with Targeted Training, Simulation 743:
    Average talk duration: 7.15 minutes
    Answer rate: 93.88%

Scenario: Varying Answer Rate with Targeted Training, Simulation 744:
    Average talk duration: 7.18 minutes
    Answer rate: 94.28%

Scenario: Varying Answer Rate with Targeted Training, Simulation 745:
    Average talk duration: 7.22 minutes
    Answer rate: 93.92%

Scenario: Varying Answer Rate with Targeted Training, S

Scenario: Varying Answer Rate with Targeted Training, Simulation 800:
    Average talk duration: 7.26 minutes
    Answer rate: 93.51%

Scenario: Varying Answer Rate with Targeted Training, Simulation 801:
    Average talk duration: 7.26 minutes
    Answer rate: 93.83%

Scenario: Varying Answer Rate with Targeted Training, Simulation 802:
    Average talk duration: 7.34 minutes
    Answer rate: 93.77%

Scenario: Varying Answer Rate with Targeted Training, Simulation 803:
    Average talk duration: 7.26 minutes
    Answer rate: 93.90%

Scenario: Varying Answer Rate with Targeted Training, Simulation 804:
    Average talk duration: 7.25 minutes
    Answer rate: 94.39%

Scenario: Varying Answer Rate with Targeted Training, Simulation 805:
    Average talk duration: 7.32 minutes
    Answer rate: 93.68%

Scenario: Varying Answer Rate with Targeted Training, Simulation 806:
    Average talk duration: 7.19 minutes
    Answer rate: 94.19%

Scenario: Varying Answer Rate with Targeted Training, S

Scenario: Varying Answer Rate with Targeted Training, Simulation 861:
    Average talk duration: 7.25 minutes
    Answer rate: 93.77%

Scenario: Varying Answer Rate with Targeted Training, Simulation 862:
    Average talk duration: 7.29 minutes
    Answer rate: 93.89%

Scenario: Varying Answer Rate with Targeted Training, Simulation 863:
    Average talk duration: 7.27 minutes
    Answer rate: 93.47%

Scenario: Varying Answer Rate with Targeted Training, Simulation 864:
    Average talk duration: 7.18 minutes
    Answer rate: 94.15%

Scenario: Varying Answer Rate with Targeted Training, Simulation 865:
    Average talk duration: 7.15 minutes
    Answer rate: 94.06%

Scenario: Varying Answer Rate with Targeted Training, Simulation 866:
    Average talk duration: 7.21 minutes
    Answer rate: 94.33%

Scenario: Varying Answer Rate with Targeted Training, Simulation 867:
    Average talk duration: 7.09 minutes
    Answer rate: 94.00%

Scenario: Varying Answer Rate with Targeted Training, S

Scenario: Varying Answer Rate with Targeted Training, Simulation 922:
    Average talk duration: 7.18 minutes
    Answer rate: 94.04%

Scenario: Varying Answer Rate with Targeted Training, Simulation 923:
    Average talk duration: 7.36 minutes
    Answer rate: 93.56%

Scenario: Varying Answer Rate with Targeted Training, Simulation 924:
    Average talk duration: 7.22 minutes
    Answer rate: 93.83%

Scenario: Varying Answer Rate with Targeted Training, Simulation 925:
    Average talk duration: 7.15 minutes
    Answer rate: 94.03%

Scenario: Varying Answer Rate with Targeted Training, Simulation 926:
    Average talk duration: 7.20 minutes
    Answer rate: 93.65%

Scenario: Varying Answer Rate with Targeted Training, Simulation 927:
    Average talk duration: 7.18 minutes
    Answer rate: 94.47%

Scenario: Varying Answer Rate with Targeted Training, Simulation 928:
    Average talk duration: 7.12 minutes
    Answer rate: 94.15%

Scenario: Varying Answer Rate with Targeted Training, S

Scenario: Varying Answer Rate with Targeted Training, Simulation 983:
    Average talk duration: 7.27 minutes
    Answer rate: 94.16%

Scenario: Varying Answer Rate with Targeted Training, Simulation 984:
    Average talk duration: 7.28 minutes
    Answer rate: 94.18%

Scenario: Varying Answer Rate with Targeted Training, Simulation 985:
    Average talk duration: 7.30 minutes
    Answer rate: 93.95%

Scenario: Varying Answer Rate with Targeted Training, Simulation 986:
    Average talk duration: 7.17 minutes
    Answer rate: 93.96%

Scenario: Varying Answer Rate with Targeted Training, Simulation 987:
    Average talk duration: 7.17 minutes
    Answer rate: 94.22%

Scenario: Varying Answer Rate with Targeted Training, Simulation 988:
    Average talk duration: 7.35 minutes
    Answer rate: 93.80%

Scenario: Varying Answer Rate with Targeted Training, Simulation 989:
    Average talk duration: 7.14 minutes
    Answer rate: 94.25%

Scenario: Varying Answer Rate with Targeted Training, S

In [20]:
# Helper function to convert decimal minutes to "minutes:seconds" format
def minutes_to_min_sec(decimal_minutes):
    minutes = int(decimal_minutes)
    seconds = int((decimal_minutes - minutes) * 60)
    return f"{minutes}:{seconds:02}"

# Analyze the results
analysis = {}

for scenario, results in all_results_reduced.items():
    #avg_wait_times = [result['average_wait_time'].total_seconds() for result in results]
    avg_talk_durations = [result['average_talk_duration'] for result in results]
    avg_answer_rates = [result['answer_rate'] for result in results]
    
    analysis[scenario] = {
        #"Average Wait Time/day": minutes_to_min_sec(np.mean(avg_wait_times)/60),
        #"Std Dev Wait Time": minutes_to_min_sec(np.std(avg_wait_times)/60),
        "Average Talk Duration/day": minutes_to_min_sec(np.mean(avg_talk_durations)/60),
        "Std Dev Talk Duration": minutes_to_min_sec(np.std(avg_talk_durations)/60),
        "Average Answer Rate/day": f"{np.mean(avg_answer_rates)*100:.2f}%",
        "Std Dev Answer Rate": f"{np.std(avg_answer_rates)*100:.2f}%"
    }

df = pd.DataFrame(analysis).T

df.head()

,Average Talk Duration/day,Std Dev Talk Duration,Average Answer Rate/day,Std Dev Answer Rate
Constant Answer Rate with Standard,7:21,0:02,93.01%,0.19%
Constant Answer Rate with Targeted Training,7:13,0:03,93.83%,0.30%
Varying Answer Rate with Standard,7:21,0:02,93.11%,0.20%
Varying Answer Rate with Targeted Training,7:13,0:03,93.90%,0.28%
